In [1]:
%load_ext autoreload
%autoreload 2
import json
from gensim.summarization.bm25 import BM25
from gensim.models.phrases import Phrases, Phraser
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 1000)
import numpy as np
from multiprocessing import Pool
from functools import partial
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
sns.set()


from tqdm.notebook import tqdm
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    tqdm.pandas()

import sys
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)


from tfidf_retrieval import TfidfRetrieval
from evaluation import *
from preprocessing import apply_pipeline, Corpus, BasicPreprocessing, BigramPreprocessor, SpacyPreprocessor

In [3]:
bm25_model = TfidfRetrieval.load("../data/models/tfidf/bm25_lemmatized_bigram.model")
# bm25_qe_model = TfidfRetrieval.load("../data/models/tfidf/bm25_lemmatized_bigram_qe.model")
# fasttext_model = WordEmbeddingRetrieval.load("../data/models/fasttext/fasttext_w29_s400_tfidf.model")
# fasttext_qe_model = WordEmbeddingRetrieval.load("../data/models/fasttext/qe_best_fasttext_exponce.model")

In [20]:
with open("../data/kit_expert_2017_paper_info.json") as file:
    paper_info = json.load(file)

In [44]:
with open("../data/kit_expert_2017_author_info.json") as file:
    author_info = json.load(file)

In [111]:
def get_experts(query, document_retrieval):
    relevant_docs = document_retrieval.get_ranked_documents(query)
    relevant_docs = relevant_docs[relevant_docs["score"] > 0.05]
    experts = {}
    for i, row in relevant_docs.iterrows():
        paper_id = str(int(row["id"]))
        authors = paper_info[paper_id]["authors"]
        for author in authors:
#             if author["id"] == "2061074560":
#                 print(paper_id, row["score"])
            experts[author["id"]] = experts.get(author["id"],0) + row["score"] / author["order_in_paper"]
    result = pd.DataFrame(experts.items(), columns=["expert_id", "score"])
    result["expert_name"] = result["expert_id"].apply(lambda expert_id: author_info[expert_id]["name"])
    return result.sort_values(by="score", ascending=False)
    

In [112]:
get_experts("natural language processing", bm25_model).head(20)

,expert_id,score,expert_name
3,2074083528,0.626569,walter f tichy
4,468492111,0.388172,mathias landhauser
2,2050175453,0.343520,achim rettinger
16,2294519613,0.309058,alexander wachtel
0,268444295,0.305100,alexander maedche
10,2231152044,0.272852,sebastian weigelt
7,2098806103,0.262097,eunah cho
9,280384801,0.218989,sebastian stuker
8,2061074560,0.196202,alex waibel
5,2105886198,0.170442,rudi studer
